# HistoryLogger 

## 설명
- 사용처
    - 히스토리를 로그파일에 쌓고 싶을때 사용하면됩니다.
- 이것이 주는 이점
    - 커널에 의존하지 않고 히스토리를 유지 가능.
- 원리
    - 에폭이 끝날때 히스토리 관련 로그를 로그파일에 추가한다.

# 사용법

## HistoryLogger 콜백 생성 후 모델 체크포인트 콜백과 함께 훈련하기

```python 
from aiffel7.historylogger import HistoryLogger ,history_graph


log_file = 'training_history.log'
history_logger = HistoryLogger(log_file)

# 모델 체크포인트 설정
checkpoint_path = "model_checkpoint.h5" # 저장할 파일명과 경로
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_best_only=True,
                             verbose=1,
                             monitor='val_loss',
                             mode='min')

# 모델 학습
history_cam_model = cam_model.fit(
    ds_train_norm,
    steps_per_epoch=int(ds_info.splits['train'].num_examples/16),
    validation_steps=int(ds_info.splits['test'].num_examples/16),
    epochs=20,
    validation_data=ds_val_norm,
    verbose=1,
    use_multiprocessing=True,
    callbacks=[checkpoint, history_logger] # 콜백 리스트에 체크포인트 추가
    ```

## 만약 하다가 메모리 부족같은 걸로 중단 되었다면 모델 체크포인트로 다시로드하고 해당 로그파일을 할당한 HistoryLogger와 함께 다시 훈련 시키기

```python

# 가중치 파일 경로
weights_path = "mix_resnet50.h5"
mix_resnet50 = keras.models.Sequential([
    keras.applications.resnet.ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(224,224,3),
        pooling='avg',
    ),
    keras.layers.Dense(num_classes, activation='softmax')
])
# 가중치 로드
mix_resnet50.load_weights(weights_path)

mix_resnet50.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01),
    metrics=['accuracy'],
)

history_resnet50_mix = mix_resnet50.fit(
    ds_train_aug_mixup, # augmentation 적용한 데이터셋 사용
    steps_per_epoch=int(ds_info.splits['train'].num_examples/16),
    validation_steps=int(ds_info.splits['test'].num_examples/16),
    epochs=20,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
    callbacks=[checkpoint,history_logger,tensorboard_callback]
)
```

## History 그래프 그리기

```python
import json

log_file = 'training_history.log'  # 이전 단계에서 사용한 로그 파일 이름

# 로그 파일에서 히스토리 데이터 로드
with open(log_file, 'r') as f:
    history = json.load(f)
    ```


```python
history_graph(history)
```